# Exploring Projec1 ideas

In [ ]:
# Dependencies
import pandas as pd
import numpy as np
import requests
import json
from pprint import pprint
import matplotlib.pyplot as plt
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')
import scipy.stats as st
from scipy.stats import linregress

# Import API key
from api_keys import geoapify_key

In [ ]:
#linear regression function
def get_regress(x,y, title, xlabel, ylabel):
   plt.scatter(x=x,y=y)
   (slope, intercept, rvalue, pvalue, stderr) = linregress(x, y)
   regress_values = x * slope + intercept
   line_eq = f'y = {round(slope,2)}x + {round(intercept,2)}'
   plt.plot(x,regress_values, color='red')
   plt.title(title)
   plt.xlabel(xlabel)
   plt.ylabel(ylabel)
   if sum(x) > 0:
      plt.annotate(line_eq,(min(x),min(y)),fontsize=15,color='red')
   else: 
      plt.annotate(line_eq,(min(x),min(y)),fontsize=15,color='red')
   print(f'The r-squared is: {rvalue**2}')
   print(f'The r-value is {rvalue}')
   print(f'The p-value is {pvalue}')
   print(f'The stderr is {stderr}')
   
   plt.show()

   

## DataFrame Preparation

In [ ]:
# Import cities file as DataFrame
mp_df =pd.read_csv("../Resources/chicago_public_health.csv")

raw_df = mp_df.copy()
mp_df.head()


In [ ]:
# List of Regions with corresponding Community areas for data manipulation
RA=[{"Far North Side":["Rogers Park","West Ridge","Uptown","Lincoln Square","O'Hare","Edgewater","North Park","Albany Park","Edison Park","Norwood Park","Jefferson Park","Forest Glen"]},
{"Northwest Side":["North Center","Lake View","Lincoln Park","Avondale","Logan Square","Hermosa","Portage Park","Irving Park","Dunning","Montclaire","Belmont Cragin"]},
{"North Side":["Near North Side","West Town","Humboldt Park","Austin"]},
{"Central":["Loop","Near South Side","Near West Side"]},
{"West Side":["East Garfield Park","West Garfield Park","North Lawndale","South Lawndale","Lower West Side"]},
{"South Side":["Greater Grand Crossing","Armour Square","Douglas","Oakland","Fuller Park","Grand Boulevard","Kenwood","Washington Park","Hyde Park","Woodlawn","South Shore"]},
{"Southwest Side":["Bridgeport","New City","West Elsdon","Gage Park","Brighton Park","McKinley Park","Archer Heights","West Lawn","Chicago Lawn","Garfield Ridge","Clearing","West Englewood","Englewood"]},
{"Far Southwest Side":["Washington Heights","Auburn Gresham","Ashburn","Beverly","Mount Greenwood","Morgan Park"]},
{"Far Southeast Side":["Chatham","Avalon Park","South Chicago","Burnside","Calumet Heights","Roseland","Pullman","South Deering","East Side","West Pullman","Riverdale","Hegewisch"]}]

#Add a new column for Region
raw_df["Region"]=""

#Populate the new column - Region

for i in range(mp_df.shape[0]):
    cma=mp_df.loc[i,"Community Area Name"]
    for r in range(len(RA)):
        k=list(RA[r])[0]
        if cma in RA[r][k]:
            mp_df.loc[i,"Region"]=k
            #print(cma,k)
            
      
        

mp_df.head()

In [ ]:
column_to_move = mp_df.pop('Region')
mp_df.insert(2, 'Region', column_to_move)


In [ ]:
pd.set_option('display.max_columns', None)
mp_df.head()


In [ ]:
lead_poison_df = raw_df.loc[(~raw_df['Childhood Lead Poisoning'].isna()) & (~raw_df['Below Poverty Level'].isna())]

lead_poison_df = lead_poison_df.sort_values(by='Community Area Name').reset_index()





In [ ]:
#Do higher rates of poverty/lower average income result in higher prevalence of childhood lead poisoning?
get_regress(lead_poison_df['Below Poverty Level'],lead_poison_df['Childhood Lead Poisoning'],'Test','Below Poverty Level','Lead Poisoning Rates')


In [ ]:
#Do certain socioeconomic brackets have a higher risk of certain health markers? Cancer/Cardiovascular Disease/Diabetes
disease_df = raw_df.loc[(~raw_df['Below Poverty Level'].isna()) & (~raw_df['Diabetes-related'].isna())]



get_regress(disease_df['Below Poverty Level'], disease_df['Diabetes-related'],'test','Poverty','Diabetes')

In [ ]:
#Do certain socioeconomic brackets have a higher risk of certain health markers? Cancer/Cardiovascular Disease/Diabetes
disease_df = raw_df.loc[(~raw_df['Per Capita Income'].isna()) & (~raw_df['Diabetes-related'].isna())]



get_regress(disease_df['Per Capita Income'], disease_df['Diabetes-related'],'test','Per Capita','Diabetes')

In [ ]:
mp_df.groupby("Region")[["Tuberculosis","Crowded Housing"]].mean()

In [ ]:
Region_df= mp_df.groupby("Region")[["Teen Birth Rate","No High School Diploma"]].mean()
Region_TPE= Region_df.reset_index()
Region_TPE.sort_values(by="No High School Diploma", inplace= True, ascending=False)
Region_TPE.reset_index().drop("index",axis=1)

In [ ]:
# Build scatter plot for Teen Birth Rate  vs Lack of High school diploma
x_values = Region_df['Teen Birth Rate']
y_values = Region_df['No High School Diploma']
plt.scatter(x_values,y_values, edgecolor="black")


# Incorporate the other graph properties
plt.xlabel('Avg. % Teen Birth Rate per 1,000 women aged 15-19')
plt.ylabel('Avg. %_aged_25_without_high_school_diploma')
plt.title(f"Teen Birth Rate  vs No High School Diploma")
plt.grid()


# Save the figure
#plt.savefig("output_data/Fig1.png")

# Show plot
plt.show()

In [ ]:

Region_TPE.plot(kind="bar",x="Region",y=["Teen Birth Rate","No High School Diploma"])
plt.show()


In [ ]:
#mp_df.head()
WestSide_TPE= mp_df.loc[mp_df["Region"]=="West Side"][["Community Area Name","Teen Birth Rate","No High School Diploma"]]
WestSide_TPE.reset_index().drop("index",axis=1)
WestSide_TPE.plot(kind="bar",x="Community Area Name",y=["Teen Birth Rate","No High School Diploma"])